In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
from SentimentAnalyser import SentimentAnalyser
from nltk.tokenize.treebank import TreebankWordDetokenizer

## Read in data 

### Market data

In [2]:
btc_market = pd.read_csv("data/BTC-USD-80.csv")
btc_market.head()

,time,close,high,low,open,volumefrom,volumeto,mid
0,1582984800,8645.88,8678.25,8606.31,8676.27,954.50,8238841.34,8642.280
1,1582988400,8693.73,8708.76,8619.73,8645.88,785.84,6824127.12,8664.245
2,1582992000,8638.05,8702.50,8630.15,8693.73,761.92,6629631.43,8666.325
3,1582995600,8701.21,8702.40,8623.94,8638.05,951.78,8259525.21,8663.170
4,1582999200,8695.56,8714.49,8666.65,8701.21,556.50,4862181.37,8690.570


#### Convert unix code time to more readable datetime format

In [3]:
btc_market['time_formatted'] = pd.to_datetime(btc_market['time'], unit='s')

In [4]:
btc_market[['time', 'time_formatted']].head()

,time,time_formatted
0,1582984800,2020-02-29 14:00:00
1,1582988400,2020-02-29 15:00:00
2,1582992000,2020-02-29 16:00:00
3,1582995600,2020-02-29 17:00:00
4,1582999200,2020-02-29 18:00:00


### Volatility data

In [5]:
btc_volatility = pd.read_csv("data/vcrix.csv")
btc_volatility.tail()

,date,vcrix
147,2020-05-27,873.952874
148,2020-05-28,871.938590
149,2020-05-29,900.197688
150,2020-05-30,898.384012
151,2020-05-31,819.182235


In [6]:
btc_volatility['date'] = pd.to_datetime(btc_volatility['date'])

In [7]:
# btc_volatility.drop(columns=['Unnamed: 0', 'n'], inplace=True)

# btc_volatility = btc_volatility[(btc_volatility['date'] >= '2020-01-01') & (btc_volatility['date'] < '2020-0-01')]
# btc_volatility

In [8]:
btc_volatility['vcrix_log_diff'] = np.log(btc_volatility['vcrix']) - np.log(btc_volatility['vcrix'].shift(1))

### Trend

In [9]:
btc_trend = pd.read_csv('data/BTC_trend.csv')
btc_trend

,date,Bitcoin,BTC,isPartial
0,2020-01-01 00:00:00,49,13,False
1,2020-01-01 01:00:00,46,13,False
2,2020-01-01 02:00:00,43,14,False
3,2020-01-01 03:00:00,0,0,False
4,2020-01-01 04:00:00,0,0,False
...,...,...,...,...
3641,2020-05-30 20:00:00,52,13,False
3642,2020-05-30 21:00:00,59,13,False
3643,2020-05-30 22:00:00,57,15,False
3644,2020-05-30 23:00:00,60,16,False


In [10]:
btc_trend['date'] = pd.to_datetime(btc_trend['date'])
btc_trend['date'].head()

0   2020-01-01 00:00:00
1   2020-01-01 01:00:00
2   2020-01-01 02:00:00
3   2020-01-01 03:00:00
4   2020-01-01 04:00:00
Name: date, dtype: datetime64[ns]

### Tweets

In [11]:
# sentiment analyser class
sent_analyser = SentimentAnalyser()

In [12]:
# stopwords for cleaning
stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])

In [13]:
# cleaning method for tweets
def processTweet(tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        tweet = [word for word in tweet if word not in stopwords]
        tweet = TreebankWordDetokenizer().detokenize(tweet)
        return tweet

In [14]:
def processTweets(chunk):
    processed_tweets = []
    tweet_sentiments = []
    for tweet in chunk['text']:
        if isinstance(tweet, str):
            # clean tweet
            processed_tweet = processTweet(tweet)
            
            # perform sentiment analysis on cleaned tweet using Sentiment Analyser
            tweet_sentiment = sent_analyser.sentiment_analyzer_scores(processed_tweet) # returns dictionary 

            # add to respective lists
            processed_tweets.append(processed_tweet)
            tweet_sentiments.append(tweet_sentiment)

    return (processed_tweets, pd.DataFrame(tweet_sentiments))

In [15]:
# read the large csv file with specified chunksize 
df_chunk = pd.read_csv('data/combined_tweets.csv', chunksize=50000) # add nrows=100 for testing

#df_chunk = pd.read_csv('data/by_day/tweets_btc_2020-05-12.csv', chunksize=500)

In [16]:
chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in df_chunk:  
    # drop columns where text contains nan
    chunk.dropna(subset=['text'], inplace=True)

    # init empty sentiment columns 
    chunk[['neg', 'neu', 'pos', 'compound']] = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan]], index=chunk.index)

    # perform data filtering 
    chunk_text_filtered = processTweets(chunk)

    # set filtered text and sentiments
    chunk['text'], chunk[['neg', 'neu', 'pos', 'compound']] = chunk_text_filtered
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk)
    
# concat the list into dataframe 
btc_tweets = pd.concat(chunk_list.copy())

In [17]:
btc_tweets.head()

,Unnamed: 0,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,...,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url,neg,neu,pos,compound
0,0.0,False,"['bitcoin', 'shorts', 'liquidated', 'Iran', 'W...",[],False,False,2,[],NaN,0,...,2020-01-03 23:59:55,1213248655320326144,/CryptoDivision/status/1213248655320326144,890003583231418368,TheCryptoDivision,NaN,0.147,0.853,0.000,-0.5095
1,1.0,False,"['bitcoin', 'BTC']",[],False,False,0,[],NaN,0,...,2020-01-03 23:59:47,1213248621438754816,/GetTheCrypto1/status/1213248621438754820,1105237963426816000,GetTheCrypto,NaN,0.000,1.000,0.000,0.0000
2,2.0,False,[],[],False,True,1,[],1.213121e+18,0,...,2020-01-03 23:59:23,1213248523585478656,/zhtttttty4/status/1213248523585478656,1186910416313737216,zhtttttty,NaN,0.088,0.749,0.163,0.3182
3,3.0,False,['Bitcoin'],[],False,False,4,[],NaN,0,...,2020-01-03 23:59:05,1213248448553730048,/WhaleTrades/status/1213248448553730048,968796006576947200,WhaleTrades 🐳,NaN,0.000,0.863,0.137,0.2263
4,4.0,False,"['Bitcoin', 'BTC']",[],False,False,0,[],NaN,0,...,2020-01-03 23:58:57,1213248411417370624,/reqBitcoinJS/status/1213248411417370624,1082323663238713344,Bitcoin.js //Bitcoin price every half hour,NaN,0.000,1.000,0.000,0.0000


In [18]:
btc_tweets.drop(columns='Unnamed: 0', inplace=True)

In [19]:
btc_tweets['timestamp'] = pd.to_datetime(btc_tweets['timestamp'])

In [20]:
btc_tweets.sort_values(by="timestamp", inplace=True)

In [21]:
# check if duplicate tweets exist
len(btc_tweets['tweet_id']) != len(set(btc_tweets['tweet_id']))

True

In [22]:
# drop all rows with duplicate tweet id
btc_tweets.drop_duplicates(subset ="tweet_id", inplace = True)

# check again if duplicate tweets exist
len(btc_tweets['tweet_id']) != len(set(btc_tweets['tweet_id']))

False

In [23]:
btc_tweets.shape

(585320, 25)

In [24]:
btc_tweets.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url,neg,neu,pos,compound
61616,False,['BTC'],[],False,False,2,['https://crypto.press/coins/BTC-bitcoin'],NaN,0,[],...,2020-01-01 00:00:00,1212161514226036736,/CryptoPressNews/status/1212161514226036736,24411256,Crypto Press,NaN,NaN,NaN,NaN,NaN
61615,False,"['1', 'bitcoin', 'price', 'coinmarketcap', '3V...",[],False,False,0,[],NaN,0,[],...,2020-01-01 00:00:02,1212161523247992832,/the1bitcoinbot/status/1212161523247992833,1113394645915054080,TheOneBitcoinBot ⚡,NaN,NaN,NaN,NaN,NaN
61614,True,"['Binance', 'Bittrex', 'bitcoin', 'trading', '...",['https://pbs.twimg.com/media/ENJ2e10WkAAzwgh....,True,False,1,['https://thesignal.io'],NaN,1,[],...,2020-01-01 00:00:02,1212161524254556160,/thesignalio/status/1212161524254556162,933113704366903296,The Signal,NaN,NaN,NaN,NaN,NaN
61613,False,"['ビットコイン', 'bitcoin', 'btc', 'ブロックチェーン']",[],False,False,0,[],NaN,0,[],...,2020-01-01 00:00:03,1212161528280973312,/bitcoinjp/status/1212161528280973312,1877257982,ビットコイン円相場,NaN,NaN,NaN,NaN,NaN
61612,False,"['BITCOIN', 'BTC', 'poloniex']",[],False,False,0,['https://clankapp.com/tx/btc/66c62510a27b7345...,NaN,0,[],...,2020-01-01 00:00:04,1212161530147561472,/ClankApp/status/1212161530147561473,1119277376108683264,Clank - Biggest Blockchains Transactions 🐳,NaN,NaN,NaN,NaN,NaN


## Test by day

In [25]:
btc_tweets_by_day = pd.read_json("data/by_day/tweets_btc_2020-03-13.json", encoding='utf-8')
btc_tweets_by_day.shape

(4005, 21)

In [26]:
btc_tweets_by_day.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,screen_name,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url
0,False,"[XBT, BTC, Bitcoin]",[],False,False,1,[http://www.xbt.money/new-version-of-crypto-wa...,,0,[],...,xbtmoney,New version of crypto wallet QPocket adds cold...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-03-13 23:59:59,2020-03-13 23:59:59,1238615823297593344,/xbtmoney/status/1238615823297593344,199453782,XBTmoney,
1,True,[bitcoin],[https://pbs.twimg.com/media/ETBx3sIXQAYvSTD.jpg],True,False,19,[],,3,[],...,Karalhoin,This is all I have to say given known current ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-03-13 23:59:52,2020-03-13 23:59:52,1238615793291706368,/Karalhoin/status/1238615793291706368,175960015,Vandelay BTC Industries,
2,False,"[cryptocurrency, crypto, blockchain, Bitcoin, ...",[],False,False,0,[https://www.quora.com/How-much-do-you-believe...,,0,[],...,arstech_biz,My answer to How much do you believe the idea ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-03-13 23:59:36,2020-03-13 23:59:36,1238615726379814912,/arstech_biz/status/1238615726379814912,962117889028272128,arstech.biz,
3,False,"[HODL, boomers, BTC]",[],False,True,0,[],1238608438797701121,0,"[{'screen_name': 'Bitcoin', 'user_id': '357312...",...,edja7772,"I know it is scarry! However, #HODL is the onl...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-03-13 23:59:28,2020-03-13 23:59:28,1238615695530905600,/edja7772/status/1238615695530905600,298805664,Edgar,
4,False,"[Trump, Stocks, Bitcoin, Cryptocurrencies, Dow...",[],False,False,0,[https://www.visionary-finance.com/news/Trump-...,,0,[],...,VisionaryFinanc,#Trump Declares National Emergency & #Stocks R...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-03-13 23:59:28,2020-03-13 23:59:28,1238615695371522048,/VisionaryFinanc/status/1238615695371522049,859252235804520448,VF,


### Merge two data frames

In [27]:
# first merge tweets with hourly trend data
btc_merged_hourly = pd.merge_asof(btc_tweets, btc_trend, left_on='timestamp', right_on='date', direction='backward')
btc_merged_hourly.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,username,video_url,neg,neu,pos,compound,date,Bitcoin,BTC,isPartial
0,False,['BTC'],[],False,False,2,['https://crypto.press/coins/BTC-bitcoin'],NaN,0,[],...,Crypto Press,NaN,NaN,NaN,NaN,NaN,2020-01-01,49,13,False
1,False,"['1', 'bitcoin', 'price', 'coinmarketcap', '3V...",[],False,False,0,[],NaN,0,[],...,TheOneBitcoinBot ⚡,NaN,NaN,NaN,NaN,NaN,2020-01-01,49,13,False
2,True,"['Binance', 'Bittrex', 'bitcoin', 'trading', '...",['https://pbs.twimg.com/media/ENJ2e10WkAAzwgh....,True,False,1,['https://thesignal.io'],NaN,1,[],...,The Signal,NaN,NaN,NaN,NaN,NaN,2020-01-01,49,13,False
3,False,"['ビットコイン', 'bitcoin', 'btc', 'ブロックチェーン']",[],False,False,0,[],NaN,0,[],...,ビットコイン円相場,NaN,NaN,NaN,NaN,NaN,2020-01-01,49,13,False
4,False,"['BITCOIN', 'BTC', 'poloniex']",[],False,False,0,['https://clankapp.com/tx/btc/66c62510a27b7345...,NaN,0,[],...,Clank - Biggest Blockchains Transactions 🐳,NaN,NaN,NaN,NaN,NaN,2020-01-01,49,13,False


In [28]:
btc_merged = pd.merge_asof(btc_merged_hourly, btc_volatility, left_on='timestamp', right_on='date', direction='backward', suffixes=("", "_vcrix"))

In [29]:
pd.set_option('display.max_colwidth', 50)

In [30]:
btc_merged.sample(n=12)

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,neu,pos,compound,date,Bitcoin,BTC,isPartial,date_vcrix,vcrix,vcrix_log_diff
548080,False,['Bitcoin'],[],False,False,0,['http://dlvr.it/RX6CRd'],NaN,0,[],...,NaN,NaN,NaN,2020-05-21 19:00:00,0,0,False,2020-05-21,878.953405,-0.010548
169482,False,"['fintech', 'finance', 'bitcoin']",[],False,False,0,['https://cryptonewswire.com/bitcoin-lender-bl...,NaN,0,[],...,NaN,NaN,NaN,2020-02-13 17:00:00,75,21,False,2020-02-13,665.934185,0.014281
559664,False,[],[],False,False,0,[],NaN,0,[],...,NaN,NaN,NaN,2020-05-24 17:00:00,0,0,False,2020-05-24,868.484067,-0.012106
497036,True,"['Bitcoin', 'BTC', 'crypto', 'financial', 'new...",['https://pbs.twimg.com/media/EXhwIcvXYAM8mMU....,True,False,1,[],NaN,1,[],...,NaN,NaN,NaN,2020-05-08 22:00:00,45,11,False,2020-05-08,865.331045,-0.018813
529654,True,"['investwithoropocket', 'investincrypto', 'def...",['https://pbs.twimg.com/media/EYNZOe-WoAI23v4....,False,False,5,[],NaN,0,[],...,NaN,NaN,NaN,2020-05-17 09:00:00,63,18,False,2020-05-17,924.307816,0.003084
200204,False,"['btc', 'bitcoin']",[],False,False,0,[],NaN,0,[],...,NaN,NaN,NaN,2020-02-21 12:00:00,59,18,False,2020-02-21,634.594824,0.115658
340953,False,"['Bitcoin', '24hRecap']",[],False,False,5,[],NaN,0,[],...,NaN,NaN,NaN,2020-03-29 14:00:00,75,19,False,2020-03-29,2157.917113,-0.000714
237334,False,"['Bitcoin', 'BTC']",[],True,True,20,[],1.234520e+18,1,"[{'screen_name': 'APompliano', 'user_id': '339...",...,NaN,NaN,NaN,2020-03-02 18:00:00,73,20,False,2020-03-02,676.500591,-0.006225
552012,False,"['USD', 'Dollar', 'BTC', 'ETH', 'XRP', 'BCH', ...",[],False,False,0,[],NaN,0,[],...,NaN,NaN,NaN,2020-05-22 18:00:00,0,0,False,2020-05-22,869.554520,-0.010751
303207,False,[],[],False,False,0,['https://www.youtube.com/watch?v=U0KKVlUl4dw'],NaN,0,[],...,NaN,NaN,NaN,2020-03-19 13:00:00,86,23,False,2020-03-19,2150.255622,0.011281


In [31]:
btc_merged.columns

Index(['has_media', 'hashtags', 'img_urls', 'is_replied', 'is_reply_to',
       'likes', 'links', 'parent_tweet_id', 'replies', 'reply_to_users',
       'retweets', 'screen_name', 'text', 'text_html', 'timestamp',
       'timestamp_epochs', 'tweet_id', 'tweet_url', 'user_id', 'username',
       'video_url', 'neg', 'neu', 'pos', 'compound', 'date', 'Bitcoin', 'BTC',
       'isPartial', 'date_vcrix', 'vcrix', 'vcrix_log_diff'],
      dtype='object')

In [32]:
btc_merged.tail()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,neu,pos,compound,date,Bitcoin,BTC,isPartial,date_vcrix,vcrix,vcrix_log_diff
585315,False,[],[],False,False,0,[],NaN,0,[],...,NaN,NaN,NaN,2020-05-30 23:00:00,60,16,False,2020-05-30,898.384012,-0.002017
585316,True,"['cryptocurrency', 'BTC', 'ElonMusk', 'Cryptop...",['https://pbs.twimg.com/media/EZTeMdmXYAA_tFr....,False,False,1,[],NaN,0,[],...,NaN,NaN,NaN,2020-05-30 23:00:00,60,16,False,2020-05-30,898.384012,-0.002017
585317,False,[],[],True,True,0,[],1.266831e+18,1,"[{'screen_name': 'chad_alt', 'user_id': '10271...",...,NaN,NaN,NaN,2020-05-30 23:00:00,60,16,False,2020-05-30,898.384012,-0.002017
585318,True,[],['https://pbs.twimg.com/media/EZTbwPnXQAEuDhT....,True,False,31,[],NaN,1,[],...,NaN,NaN,NaN,2020-05-30 23:00:00,60,16,False,2020-05-30,898.384012,-0.002017
585319,True,"['latest', 'features', 'CRCT', 'Crypto', 'Altc...",['https://pbs.twimg.com/media/EZTeFHNXkAADTVj....,False,False,8,['https://github.com/CircuitProject/Circuit-Pr...,NaN,0,[],...,NaN,NaN,NaN,2020-05-30 23:00:00,60,16,False,2020-05-30,898.384012,-0.002017


In [33]:
btc_merged.to_csv('data/btc_final.csv', index=False)

### Aggregation by day

In [34]:
btc_merged.columns

Index(['has_media', 'hashtags', 'img_urls', 'is_replied', 'is_reply_to',
       'likes', 'links', 'parent_tweet_id', 'replies', 'reply_to_users',
       'retweets', 'screen_name', 'text', 'text_html', 'timestamp',
       'timestamp_epochs', 'tweet_id', 'tweet_url', 'user_id', 'username',
       'video_url', 'neg', 'neu', 'pos', 'compound', 'date', 'Bitcoin', 'BTC',
       'isPartial', 'date_vcrix', 'vcrix', 'vcrix_log_diff'],
      dtype='object')

In [35]:
btc_by_day = btc_merged.resample('d', on='date').mean().dropna(how='all')

In [36]:
print(btc_merged.shape, btc_by_day.shape)

(585320, 32) (148, 19)


In [37]:
btc_merged[btc_merged['compound'].isnull()]

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,neu,pos,compound,date,Bitcoin,BTC,isPartial,date_vcrix,vcrix,vcrix_log_diff
0,False,['BTC'],[],False,False,2,['https://crypto.press/coins/BTC-bitcoin'],NaN,0,[],...,NaN,NaN,NaN,2020-01-01 00:00:00,49,13,False,2020-01-01,564.354535,NaN
1,False,"['1', 'bitcoin', 'price', 'coinmarketcap', '3V...",[],False,False,0,[],NaN,0,[],...,NaN,NaN,NaN,2020-01-01 00:00:00,49,13,False,2020-01-01,564.354535,NaN
2,True,"['Binance', 'Bittrex', 'bitcoin', 'trading', '...",['https://pbs.twimg.com/media/ENJ2e10WkAAzwgh....,True,False,1,['https://thesignal.io'],NaN,1,[],...,NaN,NaN,NaN,2020-01-01 00:00:00,49,13,False,2020-01-01,564.354535,NaN
3,False,"['ビットコイン', 'bitcoin', 'btc', 'ブロックチェーン']",[],False,False,0,[],NaN,0,[],...,NaN,NaN,NaN,2020-01-01 00:00:00,49,13,False,2020-01-01,564.354535,NaN
4,False,"['BITCOIN', 'BTC', 'poloniex']",[],False,False,0,['https://clankapp.com/tx/btc/66c62510a27b7345...,NaN,0,[],...,NaN,NaN,NaN,2020-01-01 00:00:00,49,13,False,2020-01-01,564.354535,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585315,False,[],[],False,False,0,[],NaN,0,[],...,NaN,NaN,NaN,2020-05-30 23:00:00,60,16,False,2020-05-30,898.384012,-0.002017
585316,True,"['cryptocurrency', 'BTC', 'ElonMusk', 'Cryptop...",['https://pbs.twimg.com/media/EZTeMdmXYAA_tFr....,False,False,1,[],NaN,0,[],...,NaN,NaN,NaN,2020-05-30 23:00:00,60,16,False,2020-05-30,898.384012,-0.002017
585317,False,[],[],True,True,0,[],1.266831e+18,1,"[{'screen_name': 'chad_alt', 'user_id': '10271...",...,NaN,NaN,NaN,2020-05-30 23:00:00,60,16,False,2020-05-30,898.384012,-0.002017
585318,True,[],['https://pbs.twimg.com/media/EZTbwPnXQAEuDhT....,True,False,31,[],NaN,1,[],...,NaN,NaN,NaN,2020-05-30 23:00:00,60,16,False,2020-05-30,898.384012,-0.002017


In [39]:
# print(btc_by_day.head())